# Create Tiles for Indonesia Coastline

In [5]:
import os

import geopandas as gpd
from odc.geo import BoundingBox

from coastlines.grids import INDONESIA_25, INDONESIA_CRS

os.environ["USE_PYGEOS"] = "0"

## Shows which coastlines.grids library is imported

It looks like the imported libraries are not from this repository, so we need to define the grid in this notebook and take some parameters from the existing grids.
First shows grid parameters.

In [6]:
print('INDONESIA_25:', INDONESIA_25, INDONESIA_25.origin)
print('INDONESIA_CRS:', INDONESIA_CRS)

INDONESIA_25: GridSpec(crs=ESRI:54034, tile_shape=Shape2d(x=2000, y=2000), resolution=Resolution(x=25, y=-25)) XY(x=8000000, y=-2000000)
INDONESIA_CRS: ESRI:54034


## Create Indonesia 25 km tiles

In [7]:
# Select relevant tiles and convert to Geopandas
bounds = BoundingBox(94, -11, 142, 6, crs='EPSG:4326').to_crs(INDONESIA_25.crs)
geom = INDONESIA_25.geojson(bbox=bounds)
tiles_projected_25 = gpd.GeoDataFrame.from_features(geom, crs='EPSG:4326').to_crs(
    INDONESIA_25.crs
)

## Indonesia coast tiles

Read Indonesia's 5K coastline

In [8]:
idn_coastline = gpd.read_file('../data/raw/GARISPANTAI_5K_SEPT2024.gdb')

In [9]:
if tiles_projected_25.crs != idn_coastline.crs:
    print(f'Reproject tiles to {idn_coastline.crs}') 
    tiles_idn = tiles_projected_25.to_crs(crs=str(idn_coastline.crs).upper())
else:
    print('Same CRS')
    tiles_idn = tiles_projected_25

Reproject tiles to EPSG:4326


Perform spatial join: tiles that intersect any coastline segment. Tiles id column merged into the table. 

In [10]:
# Perform spatial join: tiles that intersect any coastline segment
tiles_idn_coast = gpd.sjoin(tiles_idn, idn_coastline, predicate="intersects", how="inner")
tiles_idn_coast.head(3)

,geometry,idx,index_right,NAMOBJ,FCODE,REMARK,METADATA,SRS_ID,DTMVER,ELEVAS,KARGPN,TIPGPN,KODGPN,THNSBDATA,SBDATA,SKL,THNPBL,KET,Panjang,Shape_Length
64,"POLYGON ((122.62004 -10.91663, 122.62004 -11.3...","113,15",34112,Garis Pantai,DB02080140,Digitasi - FU,,4326,NaN,NaN,5.0,4,,2011-2014,6.0,4,2024,,12.584924,0.113479
64,"POLYGON ((122.62004 -10.91663, 122.62004 -11.3...","113,15",33889,Garis Pantai,DB02080140,Digitasi - FU,,4326,NaN,NaN,3.0,4,,2011-2014,6.0,4,2024,,0.331914,0.002994
64,"POLYGON ((122.62004 -10.91663, 122.62004 -11.3...","113,15",34113,Garis Pantai,DB02080140,Digitasi - FU,,4326,NaN,NaN,3.0,4,,2011-2014,6.0,4,2024,,0.791579,0.007135


Keep unique tiles only. Removing redundant tiles and unnecessary columns.

In [11]:
# Keep unique tiles only
tiles_idn_coast = tiles_idn.loc[tiles_idn.index.isin(tiles_idn_coast.index)]
tiles_idn_coast.head(3)

,geometry,idx
64,"POLYGON ((122.62004 -10.91663, 122.62004 -11.3...","113,15"
168,"POLYGON ((120.82341 -10.45667, 120.82341 -10.9...","109,16"
169,"POLYGON ((121.27256 -10.45667, 121.27256 -10.9...","110,16"


Filtered tiles based on coastline data.

In [12]:
tiles_idn_coast.explore()

In [13]:
# Export the results as GeoJSON
tiles_idn_coast = tiles_idn_coast.rename(columns={'idx': 'id'})
tiles_idn_coast.to_crs('EPSG:4326').to_file(
    '../data/raw/indonesia_25km_tiles_coast.geojson', driver='GeoJSON'
)